In [1]:
# import cv2

# # 입력 동영상 파일 경로
# input_video_path = 'D:/Men-in-Black/Traffic_Light/input_video(2).mp4'

# # 출력 동영상 파일 경로
# output_video_path = 'D:/Men-in-Black/Traffic_Light/output_video.mp4'

# # 입력 동영상 캡처
# cap = cv2.VideoCapture(input_video_path)

# # 입력 동영상의 프레임 크기 및 fps 가져오기
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = cap.get(cv2.CAP_PROP_FPS)

# # 출력 동영상 코덱 및 VideoWriter 설정
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 또는 'XVID'
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (640, 640))

# # 프레임을 읽어서 크기 조정 후 출력 동영상에 쓰기
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # 프레임 크기 조정
#     resized_frame = cv2.resize(frame, (640, 640))

#     # 출력 동영상에 프레임 쓰기
#     out.write(resized_frame)

# # 자원 해제
# cap.release()
# out.release()


In [2]:
# !pip install opencv-python

In [3]:
# pip install ultralytics==8.0.73

In [4]:
from IPython import display
from IPython.display import display, Image
from ultralytics import YOLO

# display.clear_output()

# YOLO 객체 생성
# yolo = YOLO()

# verify 메서드 실행
# yolo.check()

In [5]:
import os
HOME = os.getcwd()
print(HOME)

D:\Men-in-Black\Traffic_Light


In [6]:
# Pip install method (recommended)

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.73  Python-3.11.5 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 16.0 GB RAM, 644.4/931.5 GB disk)


In [7]:
import yaml
import torch
from torchvision import transforms
from PIL import Image as PILImage
from tqdm import tqdm
from PIL import Image
from ultralytics.yolo.engine.model import YOLO

In [8]:
# model = YOLO('D:/Men-in-Black/Traffic_Light/yolov8m.pt')

In [9]:
import cv2
from ultralytics.yolo.engine.model import YOLO
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# YOLO 모델 로드
path = 'D:/Men-in-Black/Traffic_Light/best_model.pt'
model = YOLO(path)

# 입력 동영상 파일 경로
input_video_path = 'D:/Men-in-Black/Traffic_Light/input_video(2).mp4'

# 출력 동영상 파일 경로
output_video_path = 'D:/Men-in-Black/Traffic_Light/output_video_119.mp4'

# 입력 동영상 캡처
cap = cv2.VideoCapture(input_video_path)

# 입력 동영상의 프레임 크기 및 fps 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 출력 동영상 코덱 및 VideoWriter 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 또는 'XVID'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (640, 640))

# 비동기 처리를 위한 ThreadPoolExecutor 설정
executor = ThreadPoolExecutor(max_workers=2)

# 초기화 최적화
ret, frame = cap.read()
resized_frame = cv2.resize(frame, (640, 640))

# 프레임을 읽어서 크기 조정 후 Object Detection 및 출력 동영상에 쓰기
def process_frame(frame):
    resized_frame = cv2.resize(frame, (640, 640))
    results = model(resized_frame)
    for det in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = det.tolist()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(resized_frame, f'{model.names[int(cls)]}: {conf:.2f}', (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    out.write(resized_frame)
    cv2.imshow('Object Detection', resized_frame)

# tqdm을 사용하여 진행 상황 표시
with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), unit='frame') as pbar:
    with executor:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # 비동기 처리를 통한 프레임 처리
            executor.submit(process_frame, frame)

            # 진행 상황 업데이트
            pbar.update(1)

            # 'q' 키를 누르면 종료
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()


  0%|                                                                                      | 0/2041 [00:00<?, ?frame/s]
Ultralytics YOLOv8.0.73  Python-3.11.5 torch-2.0.1+cpu CPU
  8%|██████                                                                      | 164/2041 [00:02<00:27, 68.32frame/s]

 10%|███████▉                                                                    | 214/2041 [00:03<00:28, 64.95frame/s]0: 640x640 (no detections), 749.3ms
Speed: 0.0ms preprocess, 749.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

 25%|██████████████████▉                                                         | 509/2041 [00:07<00:23, 65.06frame/s]0: 640x640 (no detections), 722.2ms
0: 640x640 (no detections), 722.2ms
Speed: 2.3ms preprocess, 361.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Speed: 2.3ms preprocess, 361.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


 27%|████████████████████▊                               



0: 640x640 (no detections), 697.9ms
Speed: 7.0ms preprocess, 697.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)































































0: 640x640 (no detections), 673.0ms
Speed: 3.0ms preprocess, 673.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 (no detections), 673.0ms
Speed: 1.5ms preprocess, 336.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 (no detections), 700.0ms
Speed: 3.0ms preprocess, 700.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

















































0: 640x640 (no detections), 664.0ms
Speed: 3.0ms preprocess, 664.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

























































0: 640x640 (no detections), 669.5ms
Speed: 2.0ms preprocess, 669.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)
























In [ ]:
import cv2
from ultralytics.yolo.engine.model import YOLO
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# YOLO 모델 로드
path = 'D:/Men-in-Black/Traffic_Light/best_model.pt'
model = YOLO(path)

# 입력 동영상 파일 경로
input_video_path = 'D:/Men-in-Black/Traffic_Light/input_video(2).mp4'

# 출력 동영상 파일 경로
output_video_path = 'D:/Men-in-Black/Traffic_Light/output_video_with_detection.mp4'

# 입력 동영상 캡처
cap = cv2.VideoCapture(input_video_path)

# 입력 동영상의 프레임 크기 및 fps 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 출력 동영상 코덱 및 VideoWriter 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 또는 'XVID'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (640, 640))

# 비동기 처리를 위한 ThreadPoolExecutor 설정
executor = ThreadPoolExecutor(max_workers=2)

# 초기화 최적화
ret, frame = cap.read()
resized_frame = cv2.resize(frame, (640, 640))

# tqdm을 사용하여 진행 상황 표시
pbar = tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), unit='frame')

def process_frame(frame, pbar):
    resized_frame = cv2.resize(frame, (640, 640))
    results = model(resized_frame)
    for det in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = det.tolist()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(resized_frame, f'{model.names[int(cls)]}: {conf:.2f}', (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    out.write(resized_frame)
    cv2.imshow('Object Detection', resized_frame)
    pbar.update(1)

# 비동기 처리를 통한 프레임 처리
with executor:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 비동기 처리를 통한 프레임 처리
        executor.submit(process_frame, frame, pbar)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(30) & 0xFF == ord('q'):  # 30ms 지연 (약 33 fps)
            break

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()
